In [1]:
import scipy
import threading
import time

import numpy as np

# Одноканальная СМО с отказами

Вход: простейший поток заявок с интенсивностью X.<br>
Время обслуживания — показательное с параметром Y = 1 / to6c. 

Работающий канал может время от времени выходить из строя (отказывать). <br>
Поток отказов канала — простейший с интенсивностью X.<br>
Время ремонта — показательное с параметром R = 1 / tр . 

Восстановление (ремонт) вышедшего из строя канала начинается мгновенно после его отказа;<br>
Заявка, которая обслуживалась в момент выхода канала из строя, покидает СМО необслуженной.<br>
Найти финальные вероятности состояний СМО:<br>
s0 — канал свободен;<br>
s1 — канал занят, исправен;<br>
s2 — канал ремонтируется<br>
характеристики СМО: относительную и абсолютную пропускную способность системы<br>


In [20]:
X = 1 / 2
Y = 1 / 6
R = 1 / 3

In [19]:
points = []
for i in range(10**4):
    t = scipy.stats.expon.rvs(scale=1/X)
    points.append(t)

sum(points) / len(points)

24.025186417354767

In [ ]:
M_request_time = 1 / X
M_processing_time = 1 / Y
M_break_down_time = 1 / X
M_repair_time = 1 / R

In [6]:
s0 = None
s1 = None
s2 = None
s0, s1, s2

(None, None, None)

In [3]:
q = Y / (X + Y)
A = X * q

In [4]:
q, A

(0.25, 0.125)

In [ ]:
class MyThread():
    pass

In [5]:
class Queueing_system():
    def __init__(self, X, Y, R, random_state=42) -> None:
        self.X = X
        self.Y = Y
        self.R = R
        self.random_state = random_state

        self.requests = 0
        self.break_downs = 0
        
        self.finished = 0
        self.rejected = 0

        # self.current_worker_thread = None
        self.request_terminated = False

        self.channel_is_busy_event = threading.Event()
        self.new_request_event = threading.Event()
        self.request_finished_event = threading.Event()
        self.stream_broke_event = threading.Event()

        self.LOOP_IS_RUNNING = False
        # self.threads = []
    
    def start(self):
        self.LOOP_IS_RUNNING = True

        thread = threading.Thread(target=self._loop, args=(), daemon=True)
        # self.threads.append(thread)
        thread.start()

        # TODO: ONLY ON FIRST START
        self.run_simplest_stream(self.X, self.new_request_event)
        self.run_simplest_stream(self.X, self.stream_broke_event, condition=self.channel_is_busy_event)
    
    def stop(self):
        self.LOOP_IS_RUNNING = False
        
        if self.channel_is_busy_event.is_set():
            self.rejected += 1
            self.channel_is_busy_event.clear()
    
    def run_simplest_stream(self, intensity, event, condition=None):
        def inner(intensity: float, event: threading.Event, condition, random_state):
            if condition is not None and not condition.is_set():
                print('Channel is not busy')
                return
            if condition is not None and condition.is_set():
                print('Channel is busy. Starting to evaluate break down time')
            
            while True:
                t = scipy.stats.expon.rvs(scale=1/intensity)
                time.sleep(t)
                
                event.set()
                if condition is not None:
                    # TODO: problem there, stop break down stream when repairing
                    print('event set?', event.is_set())
                    


        thread = threading.Thread(target=inner, args=(intensity, event, condition, self.random_state), daemon=True)
        # self.threads.append(thread)
        thread.start()

        return thread
    
    def run_event(self, intensity, event, reversed=False):
        def inner(intensity: float, event: threading.Event, random_state):
            t = scipy.stats.expon.rvs(scale=1/intensity)
            time.sleep(t)
            
            if reversed:
                event.clear()
            else:
                event.set()

        thread = threading.Thread(target=inner, args=(intensity, event, self.random_state), daemon=True)
        # self.threads.append(thread)
        thread.start()

        return thread
    
    def _loop(self):
        while self.LOOP_IS_RUNNING:

            # New request
            if self.new_request_event.is_set():

                if not self.channel_is_busy_event.is_set() and not self.stream_broke_event.is_set():
                    self.channel_is_busy_event.set()
                    self.run_event(self.Y, self.request_finished_event)
                else:
                    self.rejected += 1

                self.new_request_event.clear()
                self.requests += 1

            # Stream brakes
            if self.stream_broke_event.is_set():
                print('Broke')
            
            # Request finished
            if self.request_finished_event.is_set():
                pass
    

In [16]:
X = 100
Y = 0.01
R = 1 / 3

system = Queueing_system(X, Y, R, random_state=42)
system.start()

time.sleep(1)
system.stop()

print('Requests', system.requests)
print('Break downs', system.break_downs)
print('Finished', system.finished)
print('Rejected', system.rejected)

event set? True
event set? True
Channel is not busy
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
event set? True
Requests 61
Break downs 0
Finished 0
Rejected 61


In [17]:


channel_is_busy_event = threading.Event()
stream_broke_event = threading.Event()

run_simplest_stream(100, stream_broke_event, condition=channel_is_busy_event)

event set? True
event set? True
event set? True


NameError: name 'run_simplest_stream' is not defined